# Skfolio : Optimisation de Portefeuille Crypto

In [ ]:
%pip install -q skfolio ccxt pandas numpy matplotlib scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import ccxt
import time
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from skfolio.optimization import (
    HierarchicalRiskParity,
    MeanRisk
)
from skfolio import RiskMeasure
from skfolio.model_selection import (
    WalkForward,
    CombinatorialPurgedCV,
    cross_val_predict
)

# Téléchargement des Données Crypto

In [ ]:
def get_crypto_data(tickers, start_date='2020-01-01', end_date='2023-12-31'):
    """
    Télécharge les données crypto depuis Binance.
    
    Args:
        tickers: Liste de symboles comme ['BTC/USDT', 'ETH/USDT', 'SOL/USDT']
        start_date: Date de début au format 'YYYY-MM-DD'
        end_date: Date de fin au format 'YYYY-MM-DD'
    
    Returns:
        DataFrame avec les rendements journaliers (pct_change)
    """
    print(f"📥 Téléchargement depuis Binance {start_date} → {end_date}...")
    
    exchange = ccxt.binance({
        'enableRateLimit': True,
        'options': {'defaultType': 'spot'}
    })
    
    start_ts = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp() * 1000)
    end_ts = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp() * 1000)
    
    all_prices = {}
    
    for ticker in tickers:
        print(f"  Téléchargement {ticker}...")
        
        all_candles = []
        current_ts = start_ts
        
        while current_ts < end_ts:
            try:
                candles = exchange.fetch_ohlcv(
                    ticker, 
                    timeframe='1d',
                    since=current_ts,
                    limit=1000
                )
                
                if not candles:
                    break
                
                all_candles.extend(candles)
                current_ts = candles[-1][0] + 86400000 
                
                time.sleep(0.1)
                
            except Exception as e:
                print(f"    ⚠️ Erreur : {e}")
                break
        
        df = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.set_index('datetime').sort_index()
        
        symbol = ticker.split('/')[0]
        all_prices[symbol] = df['close']
    
    prices = pd.DataFrame(all_prices)
    
    # Filtrer strictement sur les dates demandées
    prices = prices[(prices.index >= start_date) & (prices.index <= end_date)]
    
    returns = prices.pct_change().dropna()
    
    print(f"✅ {len(returns)} jours téléchargés")
    
    return returns

In [ ]:
tickers = ['BTC/USDT', 'ETH/USDT', 'SOL/USDT']
# Besoin de 6 mois avant 2023 pour le train initial → juillet 2022
returns = get_crypto_data(tickers, start_date='2022-07-01', end_date='2024-12-31')

print(f"\n📊 Période totale: {returns.index[0].date()} → {returns.index[-1].date()}")
print(f"📊 Total: {len(returns)} jours")
print(f"📊 Cryptos: BTC, ETH, SOL")
returns.head()

# Section 1: Optimisation naïve (poids figés)

On va optimiser un portefeuille HRP sur **2021 (bull market)**, puis appliquer ces mêmes poids figés sur **2022 (bear market)**.

C'est l'erreur classique : optimiser sur l'historique disponible, puis garder ces poids comme s'ils étaient gravés dans le marbre.

In [ ]:
# Étape 1: Optimiser sur 2023
train_2023 = returns[returns.index.year == 2023]

portfolio_naive = HierarchicalRiskParity(
    portfolio_params={'compounded': True}
)
portfolio_naive.fit(train_2023)

# Afficher les poids optimisés
weights = portfolio_naive.weights_
print("="*60)
print("OPTIMISATION SUR 2023")
print("="*60)
print(f"\n💰 Poids optimisés sur 2023 :")
print(f"   BTC : {weights[0]:.1%}")
print(f"   ETH : {weights[1]:.1%}")
print(f"   SOL : {weights[2]:.1%}")

# Performance sur 2023
pred_train = portfolio_naive.predict(train_2023)
print(f"\n📈 Performance sur 2023 (période d'optimisation) :")
print(f"   Rendement annualisé : {pred_train.annualized_mean:.1%}")
print(f"   Max drawdown        : {pred_train.max_drawdown:.1%}")

## Maintenant, on garde les MÊMES poids et on les applique sur 2024

In [ ]:
# Étape 2: Appliquer les poids figés de 2023 sur 2024
test_2024 = returns[returns.index.year == 2024]

# On utilise le MÊME portfolio avec les poids calculés sur 2023
pred_test = portfolio_naive.predict(test_2024)

print("="*60)
print("TEST SUR 2024 - AVEC POIDS FIGÉS DE 2023")
print("="*60)
print(f"\n💰 Rappel des poids (calculés sur 2023, appliqués sur 2024) :")
print(f"   BTC : {weights[0]:.1%}")
print(f"   ETH : {weights[1]:.1%}")
print(f"   SOL : {weights[2]:.1%}")

print(f"\n📉 Performance sur 2024 (naïf) :")
print(f"   Rendement annualisé : {pred_test.annualized_mean:.1%}")
print(f"   Max drawdown        : {pred_test.max_drawdown:.1%}")

print(f"\n📊 Comparaison 2023 vs 2024 :")
print(f"   Rendement 2023 : +{pred_train.annualized_mean:.1%}")
print(f"   Rendement 2024 : {pred_test.annualized_mean:.1%}")
print(f"   Dégradation    : {pred_train.annualized_mean - pred_test.annualized_mean:.1%}")

# Section 2: Walk Forward (réajustement mensuel)

Au lieu de figer les poids, on les **réajuste tous les mois** avec Walk Forward.

**Période testée:** 2023-2024 (2 ans complets)

Fenêtres glissantes : **6 mois d'entraînement, 1 mois de test**

Les poids s'adaptent progressivement aux changements de marché.

In [ ]:
returns_wf = returns

cv_wf = WalkForward(train_size=180, test_size=30, freq='D')

portfolio_wf = HierarchicalRiskParity()
pred_wf = cross_val_predict(portfolio_wf, X=returns_wf, cv=cv_wf,
                            portfolio_params={'compounded': True})

print("="*60)
print("WALK FORWARD : Réajustement mensuel (2023-2024)")
print("="*60)
print(f"   Fenêtre train : 180 jours (~6 mois)")
print(f"   Fenêtre test  : 30 jours (~1 mois)")
print(f"   Périodes      : {len(pred_wf.portfolios)}")

print(f"\n📉 Performance globale Walk Forward (out-of-sample) :")
print(f"   Rendement annualisé : {pred_wf.annualized_mean:.1%}")
print(f"   Max drawdown        : {pred_wf.max_drawdown:.1%}")

print(f"\n✅ COMPARAISON COMPLÈTE Naïf vs Walk Forward :")
print(f"\n   RENDEMENTS:")
print(f"      Naïf 2024           : {pred_test.annualized_mean:.1%}")
print(f"      Walk Forward 2023-24: {pred_wf.annualized_mean:.1%}")
print(f"      Différence          : {pred_wf.annualized_mean - pred_test.annualized_mean:.1%}")
print(f"\n   DRAWDOWNS:")
print(f"      Naïf 2024           : {pred_test.max_drawdown:.1%}")
print(f"      Walk Forward 2023-24: {pred_wf.max_drawdown:.1%}")
print(f"      Protection          : {pred_test.max_drawdown - pred_wf.max_drawdown:.1%}")

# Section 3: CombinatorialPurgedCV - Mesurer la robustesse

**Problème avec Walk Forward:** On obtient UN seul résultat. Si on a eu de la chance (ou malchance) sur le timing, ça biaise la conclusion.

**Solution:** Tester plusieurs scénarios différents de périodes train/test pour obtenir une **distribution** de résultats au lieu d'un seul chiffre.

**Interprétation:**
- Si tous les scénarios donnent des résultats similaires → Stratégie robuste
- Si les résultats varient énormément (écart >20%) → Stratégie fragile, dépend du timing

In [ ]:
# CombinatorialPurgedCV avec purging et embargoing
cv_comb = CombinatorialPurgedCV(
    n_folds=10,
    n_test_folds=2,
    purged_size=5,   # Purge de 5 jours
    embargo_size=2  # Embargo de 2 jours
)

# Génération de la population de portefeuilles
portfolio_comb = HierarchicalRiskParity()
population = cross_val_predict(portfolio_comb, X=returns, cv=cv_comb,
                               portfolio_params={'compounded': True})

# Extraction des statistiques
perf_list = [p.annualized_mean for p in population]
dd_list = [p.max_drawdown for p in population]

print("="*60)
print("COMBINATORIALPURGEDCV - Test de robustesse")
print("="*60)
print(f"\n🎲 Nombre de scénarios testés : {len(perf_list)}")
print(f"   (différentes combinaisons de périodes train/test)")

print(f"\n📊 RENDEMENT ANNUALISÉ :")
print(f"   Résultat typique (médiane) : {np.median(perf_list):.1%}")
print(f"   Pire cas                   : {np.min(perf_list):.1%}")
print(f"   Meilleur cas               : {np.max(perf_list):.1%}")
print(f"   Écart (max - min)          : {np.max(perf_list) - np.min(perf_list):.1%}")

print(f"\n📊 MAX DRAWDOWN :")
print(f"   Drawdown typique (médiane) : {np.median(dd_list):.1%}")
print(f"   Pire cas                   : {np.max(dd_list):.1%}")
print(f"   Meilleur cas               : {np.min(dd_list):.1%}")
print(f"   Écart (max - min)          : {np.max(dd_list) - np.min(dd_list):.1%}")

print(f"\n💡 CONCLUSION :")
ecart_rendement = np.max(perf_list) - np.min(perf_list)
if ecart_rendement < 10:
    print(f"   ✅ Stratégie ROBUSTE (écart de {ecart_rendement:.1%} seulement)")
elif ecart_rendement < 20:
    print(f"   ⚠️  Stratégie MODÉRÉMENT robuste (écart de {ecart_rendement:.1%})")
else:
    print(f"   ❌ Stratégie FRAGILE (écart de {ecart_rendement:.1%}, dépend du timing)")